In [59]:
import requests
import cssutils
from bs4 import BeautifulSoup
import os
import json
import pandas as pd
from pymongo import MongoClient

link = 'https://store.steampowered.com/search/results'
head = {'cookie': 'sessionid=5ae342dee8ccf71fc64e6471'}

# Game Scraping

In [60]:
def scrap_game_detail():# game_rating_descriptors, game_rating_icons, achievement_block, game_area_metascore
    datas = []

    for j in range(1, 2668):
        param = {
            'page': j,
        }
        try:
            req = requests.get(link, params=param, headers=head)
            soup = BeautifulSoup(req.text, 'html.parser')
        except:
            pass
        
        conten = soup.find('div', {'id': 'search_resultsRows'}).find_all('a')#tablodaki her bir oyun için
        for i in conten:
            #search page
            #url
            url = i['href']
            
            #title
            title = i.find('div', 'col search_name ellipsis').text.strip().replace('\n', ' ')

            #price
            try:
                price = i.find('div', 'col search_price responsive_secondrow').text.strip()
            except Exception:
                price = 'discount from ' + i.find('span', {'style': 'color: #888888;'}).text.replace(' ', '.') + ' to ' + i.find('div', 'col search_price discounted responsive_secondrow').find('br').next_sibling.strip() + f" ({i.find('div', 'col search_discount responsive_secondrow').text.replace('-', '').strip()})"
            if price == '':
                price = 'none'

            #release
            release = i.find('div', 'col search_released responsive_secondrow').text
            if release == '':
                release = 'none'

            #game page
            req_piece = requests.get(url, headers=head)
            soup_piece = BeautifulSoup(req_piece.text, 'html.parser')
            
            game_id = url.split("/")[4]

            #description
            description = soup_piece.find("div", {"class" : "game_description_snippet"})
            if description == None:
                description = None 
            else:
                description = description.text.strip().replace('\n', ' ')

            #header image
            header_image = soup_piece.find("img", {"class" : "game_header_image_full"})
            if header_image == None:
                header_image = None
            else:
                header_image = header_image['src']
            
            #developer
            try:
                developer = soup_piece.find_all("div", {"class" : "dev_row"})[0]
                if developer == None:
                    developer = None
                else:
                    developer = developer.find("a").text
            except:
                developer = None
            
            #publisher
            try:
                publisher = soup_piece.find_all("div", {"class" : "dev_row"})[1]
                if developer == None:
                    publisher = None
                else:
                    publisher = publisher.find("a").text
            except:
                publisher = None
            
            #tags
            tags = soup_piece.find("div", {"class" : "glance_tags popular_tags"})
            if tags == None:
                tags = None
            else:
                tags = [i.text.strip().replace('\n', ' ') for i in tags.find_all("a")][:5]
              
            #languages
            try:
                languages = soup_piece.find_all("div", {"id" : "languageTable"})
                if languages == None:
                    languages = None
                else:
                    languages = [i.text.strip().replace('\n', ' ') for i in languages[0].find_all("td", {"class":"ellipsis"})]
            
            except:
                languages = None
            
            #background image
            background_image = soup_piece.find("img", {"class" : "game_page_background game"})
            if background_image == None:
                background_image = None
            else:
                background_image = cssutils.parseStyle(background_image["style"])['background-image']
                background_image = background_image.replace('url(', '').replace(')', '')
                
            sys_req = soup_piece.find("div", {"class" : "game_area_sys_req sysreq_content active"})
            if sys_req == None:
                sys_req = None
            else:
                sys_req = sys_req.text.strip().replace('\n', ' ')
                
            #PEGI Rating
            rating_descriptors = soup_piece.find("div", {"class" : "game_rating_descriptors"})
            if rating_descriptors == None:
                rating_descriptors = None
            else:
                rating_descriptors = rating_descriptors.text.split(",")
                rating_descriptors = [item.strip() for item in rating_descriptors]
            
            rating_image = soup_piece.find("div", {"class" : "game_rating_icon"})
            if rating_image == None:
                rating_image = None
            else:
                rating_image = rating_image.find("img")
                if rating_image == None:
                     rating_image == None
                else:
                    rating_image = rating_image["src"]
            
            #Achievement Count
            achievement_block = soup_piece.find("div", {"id":"achievement_block"})
            if achievement_block == None:
                achievement_block = None
            else:
                achievement_block = achievement_block.find("div", {"class":"block_title"})
                if achievement_block == None:
                    achievement_block = None
                else:
                    achievement_block = achievement_block.text.strip()
            
            #Metacritic Score
            metacritic_score = soup_piece.find("div", {"id":"game_area_metascore"})
            if metacritic_score == None:
                metacritic_score = None
            else:
                metacritic_score = metacritic_score.find("div", {"class":"score high"})
                if metacritic_score == None:
                    metacritic_score = None
                else:
                    metacritic_score = metacritic_score.text.strip()
            
            #DLC
            all_dlc_list = soup_piece.find("div", {"class":"game_area_dlc_list"})
            if all_dlc_list == None:
                dlc_info = None
            else:
                all_dlc_list = all_dlc_list.find_all("a", {"class":"game_area_dlc_row odd ds_collapse_flag ds_collapse_flag_tiny"})
                dlc_name_list = [item.find("div", {"class":"game_area_dlc_name"}).text.strip() for item in all_dlc_list]
                dlc_link_list = [item['href'] for item in all_dlc_list]
                dlc_price_list = [item.find("div", {"class":"game_area_dlc_price"}).text.strip() for item in all_dlc_list]

                dlc_info = {"dlc_name_list" : dlc_name_list, "dlc_link_list" : dlc_link_list, "dlc_price_list" : dlc_price_list}
            
            

            data = {
                'id':game_id,
                'title': title,
                'description': description,
                'price': price,
                'release': release,
                'link': url,
                'header_image': header_image,
                'developer': developer,
                'publisher': publisher,
                'tags':tags,
                'languages':languages,
                'background_image': background_image,
                'sys_req':sys_req,
                'rating_descriptors':rating_descriptors,
                'rating_image':rating_image,
                'achievement_block':achievement_block,
                'metacritic_score':metacritic_score,
                'dlc_info':dlc_info
            }
            datas.append(data)
        print(j)
    return datas

In [61]:
all_game = scrap_game_detail()

KeyboardInterrupt: 